This notebook moves towards GPy and GPyOpt libraries to make use of their GP and Bayesian optimization implementations (the main goal is still to show seasonality breaks standard methods)

In [ ]:
import math
import numpy as np
import GPy
import GPyOpt
import matplotlib.pyplot as plt
from IPython import display
import time
%matplotlib inline

In [ ]:
x = np.linspace(0, 1, 1000)
plt.plot(x, np.sin(2 * math.pi * np.linspace(-0 * 1.0 / 7, 1 - 0 * 1.0 / 7, 1000)), label="0")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-1 * 1.0 / 7, 1 - 1 * 1.0 / 7, 1000)), label="1")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-2 * 1.0 / 7, 1 - 2 * 1.0 / 7, 1000)), label="2")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-3 * 1.0 / 7, 1 - 3 * 1.0 / 7, 1000)), label="3")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-4 * 1.0 / 7, 1 - 4 * 1.0 / 7, 1000)), label="4")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-5 * 1.0 / 7, 1 - 5 * 1.0 / 7, 1000)), label="5")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-6 * 1.0 / 7, 1 - 6 * 1.0 / 7, 1000)), label="6")
plt.title("Example of a moving periodic signal")
plt.legend()
plt.show()

In [ ]:
class DataGenerator:
    def __init__(self, sigma_obs):
        self.day_of_the_week = 0
        self.sigma_obs = sigma_obs

    def sample(self, x):
        # Handling cases when x is both scalar and numpy array
        self.day_of_the_week = (self.day_of_the_week + 1) % 7
        return np.sin(2 * math.pi * (x - self.day_of_the_week * 1.0 / 7)) + self.sigma_obs * np.random.randn()


In [ ]:
data_gen = DataGenerator(sigma_obs=0.1)

def f(parameters):
    parameters = parameters[0]
    x = parameters[0]
    score = -data_gen.sample(x)
    score = np.array(score)
    return score

In [ ]:
bounds = [
            {'name': 'x', 'type': 'continuous', 'domain': (0, 1)}
         ]

In [ ]:
np.random.seed(777)
optimizer = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds,
                                                acquisition_type ='MPI',
                                                acquisition_par = 0.1,
                                                exact_eval=True)

In [ ]:
max_iter = 50
max_time = 60
optimizer.run_optimization(max_iter, max_time)

In [ ]:
plt.scatter(optimizer.X[:, 0], optimizer.Y)
plt.title("Sampled points during standard Bayesian optimization")
plt.show()

In [ ]:
kernel = GPy.kern.RBF(input_dim=1)
model = GPy.models.GPRegression(optimizer.X, optimizer.Y, kernel)
model.optimize()

model.plot()
plt.show()